# 04. Semantic Kernel

In this lab, we will introduce **Semantic Kernel**. Like **Langchain**, it provides a framework for working with AI models and supports applications written in Python. Semantic Kernel also adds support for .NET and Java applications.

As with the other labs, we'll start by reading in values from the `.env` file.

___
**NOTE**: This lab uses **.NET** so be sure to select .NET and not Python when starting to use this notebook.
___

In [1]:
#r "nuget: dotenv.net, 3.1.2"

using dotenv.net;

// Read values from .env file
var envVars = DotEnv.Fluent()
    .WithoutExceptions()
    .WithEnvFiles("../../../.env")
    .WithTrimValues()
    .WithDefaultEncoding()
    .WithOverwriteExistingVars()
    .WithoutProbeForEnv()
    .Read();

// Load values into variables and strip quotes
var model = envVars["AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME"].Replace("\"", "");
var azureEndpoint = envVars["AZURE_OPENAI_ENDPOINT"].Replace("\"", "");
var apiKey = envVars["AZURE_OPENAI_API_KEY"].Replace("\"", "");


Installed Packages dotenv.net, 3.1.2

To start working with Semantic Kernel, we'll need to load its nuget package.

In [2]:
#r "nuget: Microsoft.SemanticKernel, 1.0.1"

Installed Packages Microsoft.SemanticKernel, 1.0.1

Semantic Kernel works by creating an instance of the Kernel and then adding in various plugins to perform different functions. Those addins or functions can then be called individually or chained together to perform more complex tasks.

We use the standard .NET `builder` pattern to initialise the kernel. Notice that we pass in the details of the completion model that we're going to use, the Azure OpenAI API endpoint URL and the API key.

In [3]:
using Microsoft.SemanticKernel;

var builder = Kernel.CreateBuilder();
builder.Services.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
var kernel = builder.Build();

Let's create a Semantic Function to perform a simple request to Azure OpenAI. In this case, the function contains a *prompt template*. The template allows us to define a prompt and add placeholders for values that we will provide later. These values could come from user input, or another function, for example.

In [4]:
var whatCanIMakeFunction = kernel.CreateFunctionFromPrompt(
    new PromptTemplateConfig()
    {
        Template = @"What interesting things can I make with a {{$item}}?",
        InputVariables = [
            new() { Name = "item", Description = "An item to make something with.", IsRequired=true }
        ]
    });


`{{$item}}` represents the value we will provide later and will be replaced when we make the call to Azure OpenAI.

Next, we'll define a value for `item` and then call the function, passing in the `item` as we do so.

In [5]:
string item = "raspberry pi";

var response = await kernel.InvokeAsync(whatCanIMakeFunction, new () { { "item", item }});

Console.WriteLine(response);

The Raspberry Pi is a versatile and powerful single-board computer that can be used for a wide range of projects. Here are some interesting things you can make with a Raspberry Pi:

1. Retro Gaming Console: Set up a Raspberry Pi as a retro gaming console using emulation software like RetroPie or Lakka, and play classic games from systems like NES, SNES, Sega Genesis, etc.

2. Home Media Center: Turn your Raspberry Pi into a media center using Kodi or Plex, and stream movies, TV shows, and music to your TV or any other device.

3. Smart Home Hub: Create a smart home hub using Home Assistant or OpenHAB, and control your lights, appliances, security cameras, and more from a single interface.

4. Portable Arcade Machine: Build a portable arcade machine with a Raspberry Pi and a small display, and play your favorite arcade games on the go.

5. Weather Station: Set up a Raspberry Pi as a weather station to monitor temperature, humidity, air pressure, and other weather conditions using sensor

The result is a list of ideas for things we could make with a Raspberry Pi. 

We could then use Semantic Kernel to take that list of ideas and summarize it. First, we'll create a new prompt that will generate a summary of some text.

In [6]:
var thingsToMakeSummary = kernel.CreateFunctionFromPrompt(
    new PromptTemplateConfig()
    {
        Template = @"Summarize the following text: {{$thingsToMake}}?",
        InputVariables = [
            new() { Name = "thingsToMake", Description = "A list of things you could make.", IsRequired=true }
        ]
    });

Next, we can call the kernel again, this time passing in the list of ideas, the `response` from the previous call, as the value for `thingsToMake`.

In [7]:
var summary = await kernel.InvokeAsync(thingsToMakeSummary, new () { { "thingsToMake", response }});

Console.WriteLine(summary);

The Raspberry Pi is a versatile single-board computer that can be used for various projects. Some examples include creating a retro gaming console, turning it into a media center, creating a smart home hub, building a portable arcade machine, setting up a weather station, automating your home, turning it into a network attached storage device, using it as a controller for robots or drones, setting it up as an IoT gateway, or building a DIY security system. The possibilities for projects with a Raspberry Pi are endless.


## Summary

Semantic Kernel is another example of an AI orchestrator. Like other orchestrators, it can be used to simplify the process of creating complex AI applications.

## Next Section

📣 [AI Orchestration](../../03-orchestration/README.md)